In this notebook, we'll experiment to see how adding more patrons to a market can change the long term outcome of the market. As usual, here are a few utility functions.

In [1]:
WEI = 1
GWEI = 10**9
WHOLE = 10**18

def wei_to_whole(amount):
    return amount // 10**18

def gwei_to_whole(amount):
    return amount // 10**9

def gwei_to_wei(amount):
    return amount * 10**9

def wei_to_gwei(amount):
    return amount // 10**9

def convert_support_price(price):
    """Support price is ETH wei / MKT gwei. Convert to ETH whole / MKT whole"""
    # ETH gwei / MKT gwei
    price_gwei_to_gwei = wei_to_gwei(price)
    # Note units cancel
    price_whole_to_whole = price_gwei_to_gwei
    return price_whole_to_whole

Here are the market parameters. The rest of the notebook is designed so that if you change these parameters, the rest of the notebook should execute correctly

In [2]:
PRICE_FLOOR = int(.001 * GWEI)
SPREAD = 110 # 10 %
LIST_REWARD = .000025 * WHOLE # MKT
STAKE = .01 * WHOLE # MKT
VOTE_BY = 7 # Days. Are units right?
BACKEND_PAYMENT = 5
MAKER_PAYMENT = 25
RESERVE_PAYMENT = 100 - BACKEND_PAYMENT - MAKER_PAYMENT
COST_PER_BYTE = 100 * GWEI # ETH

Ok, now let's define the parameters that govern the current simulation. The first cell holds the critical one with the number of patrons.

In [13]:
N_PATRONS = 5
# Support each patron provides
SUPPORT = 1000 * WHOLE # ETH

Now le's introduce various parameters.

In [5]:
CREATOR_SUPPORT = 100 * WHOLE # ETH
N_MAKERS = 1000
# This is number of listings *per* maker
N_LISTINGS_PER_MAKER = 100
# Size of an individual data purchase in ETH
PURCHASE_SIZE = 1000 * WHOLE # ETH
# Number of purchases
N_PURCHASES = 90

We have one large cell that models the entire simple market simulation.

In [24]:

def run_simulation(N_PATRONS):
  print("-------------------------------------------------------------")
  print("Running simulation with %d patrons" % N_PATRONS)
  RESERVE = 0 # Initial reserve is empty
  MARKET_TOTAL = 0 # No Market tokens at start

  # Creator does their thing
  CREATOR_BLOCK_SIZE = 1 * WHOLE # MKT
  MARKET_TOTAL += CREATOR_BLOCK_SIZE
  # Creator deposits funds in reserve
  RESERVE += CREATOR_SUPPORT
  print("MARKET_TOTAL: %d, RESERVE: %d" % (wei_to_whole(MARKET_TOTAL), wei_to_whole(RESERVE)))

  # Data Structure to keep track of coin table
  coin_table = {}
  # We're just going to use whole units for convenience
  coin_table["CREATOR"] = CREATOR_BLOCK_SIZE / 10**18

  # Now some patrons join the market.
  print("SUPPORT: %d" % wei_to_whole(SUPPORT))
  for i in range(N_PATRONS):
      print("Patron %d supports" % (i+1))
      SUPPORT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * MARKET_TOTAL))
      print("SUPPORT_PRICE (ETH whole for MKT whole): %d" % convert_support_price(SUPPORT_PRICE))
      MINTED = (SUPPORT // SUPPORT_PRICE) * GWEI # Units of WEI
      print("MINTED: %f" % (MINTED / 10**18))
      MARKET_TOTAL += MINTED
      RESERVE += SUPPORT
      coin_table["PATRON_%d" % (i+1)] = MINTED / 10**18

  # Now let's add some makers
  for i in range(N_MAKERS):
      for j in range(N_LISTINGS_PER_MAKER):
          MAKER = "MAKER_%d" % (i+1)
          MINTED = LIST_REWARD
          MARKET_TOTAL += MINTED
          if MAKER not in coin_table:
              coin_table[MAKER] = 0
          coin_table[MAKER] += MINTED/10**18

  # Now let's add some purchases
  BACKEND_REVENUE = 0
  for n in range(N_PURCHASES):
      #print("Purchase %d" % n)
      PURCHASE = PURCHASE_SIZE
      BACKEND_SHARE = (PURCHASE * BACKEND_PAYMENT)//100
      MAKER_SHARE = (PURCHASE * MAKER_PAYMENT)//100
      RESERVE_SHARE = (PURCHASE * RESERVE_PAYMENT)//100
      # Pay out the reserve share
      RESERVE += RESERVE_SHARE
      # Backend share is paid directly to backend
      BACKEND_REVENUE += BACKEND_SHARE
      # The maker share is created via the support() function. Let's find the current support price:
      SUPPORT_PRICE = PRICE_FLOOR + ((SPREAD * RESERVE * GWEI) // (100 * MARKET_TOTAL))
      #print("SUPPORT_PRICE (ETH whole for MKT whole): %d" % convert_support_price(SUPPORT_PRICE))
      # Now mint the makers their access rewards
      ONE_MAKER_SHARE = MAKER_SHARE // N_MAKERS
      MINTED = (ONE_MAKER_SHARE // SUPPORT_PRICE) * GWEI # Units of WEI
    
  print("#####################")
  print("MARKET_TOTAL: %f" % (MARKET_TOTAL / 10**18))
  print("RESERVE: %f" % (RESERVE / 10**18))
  CREATOR_ETH = (coin_table["CREATOR"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
  print("CREATOR ETH: %f" % CREATOR_ETH)
  CREATOR_PROFIT = CREATOR_ETH - (CREATOR_SUPPORT / 10**18)
  print("CREATOR PROFIT: %f" % CREATOR_PROFIT)
  # All patrons have the same share
  PATRON_ETH = (coin_table["PATRON_1"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
  print("BACKEND_REVENUE: %f" % (BACKEND_REVENUE / 10**18))
  print("PATRON ETH: %f" % PATRON_ETH)
  PATRON_PROFIT = PATRON_ETH - (SUPPORT/10**18)
  print("PATRON_PROFIT: %f" % PATRON_PROFIT)
  MAKER_ETH = (coin_table["MAKER_1"]/(MARKET_TOTAL / 10**18)) * (RESERVE / 10**18)
  print("MAKER ETH: %f" % MAKER_ETH)
  print("Ending simulation")
  print("-------------------------------------------------------------")
  return (RESERVE / 10**18)

patrons_list = range(10)
reserve_sizes = []
for n_patrons in patrons_list:
    reserve_size = run_simulation(N_PATRONS=(n_patrons+1))
    reserve_sizes.append(reserve_size)  

-------------------------------------------------------------
Running simulation with 1 patrons
MARKET_TOTAL: 1, RESERVE: 100
SUPPORT: 1000
Patron 1 supports
SUPPORT_PRICE (ETH whole for MKT whole): 110
MINTED: 9.090826
#####################
MARKET_TOTAL: 12.590826
RESERVE: 64100.000000
CREATOR ETH: 5091.008145
CREATOR PROFIT: 4991.008145
BACKEND_REVENUE: 4500.000000
PATRON ETH: 46281.471491
PATRON_PROFIT: 45281.471491
MAKER ETH: 12.727520
Ending simulation
-------------------------------------------------------------
-------------------------------------------------------------
Running simulation with 2 patrons
MARKET_TOTAL: 1, RESERVE: 100
SUPPORT: 1000
Patron 1 supports
SUPPORT_PRICE (ETH whole for MKT whole): 110
MINTED: 9.090826
Patron 2 supports
SUPPORT_PRICE (ETH whole for MKT whole): 119
MINTED: 8.339456
#####################
MARKET_TOTAL: 20.930283
RESERVE: 65100.000000
CREATOR ETH: 3110.325854
CREATOR PROFIT: 3010.325854
BACKEND_REVENUE: 4500.000000
PATRON ETH: 28275.432532
P